# Stock Price Prediction Using LSTM


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Loading Dataset


In [ ]:
data = pd.read_csv('GOOG.csv', date_parser=True)

In [ ]:
data_training = data[data['Date']< '2019-01-01'].copy()

In [ ]:
data_test = data[data['Date']>= '2019-01-01'].copy()

In [ ]:
training_data = data_training.drop(['Date', 'Adj Close'], axis=1)

# Feature Scaling

In [ ]:
sc = MinMaxScaler(feature_range = (0, 1))
training_data_scaled = sc.fit_transform(training_data)

# Creating Data With TimeStamps

In [ ]:
X_train = []
y_train = []
for i in range(90, training_data_scaled.shape[0]):
    X_train.append(training_data_scaled[i-90:i, 0])
    y_train.append(training_data_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Building The LSTM 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units = 100, activation='relu', return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.25))

regressor.add(LSTM(units = 140, activation='relu', return_sequences = True))
regressor.add(Dropout(0.35))

regressor.add(LSTM(units = 90, activation='relu', return_sequences = True))
regressor.add(Dropout(0.35))

regressor.add(LSTM(units = 120))
regressor.add(Dropout(0.25))

regressor.add(Dense(units = 1))

In [ ]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(X_train, y_train, epochs = 11, batch_size = 32 )

# Prepare Test Dataset

In [ ]:
past_90_days=data_training.tail(90)

In [ ]:
df= past_90_days.append(data_test, ignore_index= True )

In [ ]:
df= df.drop(['Date', 'Adj Close'], axis=1)

In [ ]:
inputs= sc.transform(df)

In [ ]:
X_test = []
y_test = []
for i in range(90, inputs.shape[0]):
    X_test.append(inputs[i-90:i, 0])
    y_test.append(inputs[i, 0])
X_test, y_test = np.array(X_test), np.array(y_test)

In [ ]:
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
y_pred = regressor.predict(X_test) 

In [ ]:
scale=1/sc.scale_[0]

In [ ]:
y_pred= y_pred*scale
y_test= y_test*scale

# Plotting the Graph

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(y_test, color = 'black', label = 'Google Stock Price')
plt.plot(y_pred, color = 'green', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()

# Calculating the Error

In [ ]:
error=[]
sum_of_error=0
for i in range(len(y_pred)):
    error_percentage=100*abs(y_pred[i][0]-y_test[i])/y_test[i]
    error.append(error_percentage)
    sum_of_error+=error_percentage
avg_error=sum_of_error/len(y_pred)
avg_error

## Plotting Graph for Error

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(error, color = 'red', label = 'Error')
plt.title('Google Stock Price Prediction Error')
plt.xlabel('Days')
plt.ylabel('Error percentage')
plt.legend()
plt.show()